In [ ]:
import os, sys, subprocess, gc, traceback, torch
from pathlib import Path
import soundfile as sf
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
def log(msg): 
    ts = datetime.now().strftime("%H:%M:%S")
    line = f"[{ts}] {msg}"
    print(line, flush=True)
def setup():
    log("Checking environment...")
    if subprocess.run("sox --version", shell=True, capture_output=True).returncode != 0:
        subprocess.run("apt-get update -q && apt-get install -y -q sox libsox-fmt-all", shell=True)
    subprocess.run("pip install -q -U qwen-tts==0.1.1", shell=True)
# Parameters
text = "Wait it out end to end. Qwen GPU victory!"
voice = "Ryan"
output_file = "final_victory_gpu.wav"
try:
    setup()
    from qwen_tts import Qwen3TTSModel
    model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice", torch_dtype=torch.float32, device_map="auto")
    with torch.inference_mode():
        audio_array, sample_rate = model.generate_custom_voice(text=text, speaker="Ryan", do_sample=False, subtalker_dosample=False)
    if hasattr(audio_array, "cpu"): audio_array = audio_array.cpu().numpy()
    sf.write(output_file, audio_array.flatten(), int(sample_rate))
    log(f"SUCCESS: Audio saved to {output_file}")
except Exception as e:
    log(f"ERROR: {e}")
    traceback.print_exc()